In [2]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import random
import math
import sys
import json


if './' not in sys.path:
    sys.path.append('./')
    
from Album import *

In [14]:
table_list = ["T0", "T10", "T13"]
tables_list = []
global_lists, global_info, fake_adverse_list = {}, {}, {}
outputFile = open('results_onefake.txt', "w+")

fp = open("all_jsons.json","r")
all_jsons = json.load(fp)
for jsonfile in all_jsons["all_json"]:
    jsonfile["Title"] = jsonfile["Title"][0]
    jsonfile["Released"] = jsonfile["Released"][0]
    jsonfile["Recorded"] = jsonfile["Recorded"][0]
    jsonfile["Length"] = jsonfile["Length"][0]
    jsonfile["Tablename"] = jsonfile["Tablename"][0]
    dictionary = reformDict(jsonfile)
    dictionary["Tablename"].val = "T"+"0"+"_"+dictionary["Tablename"].val
    tables_list.append(dictionary)
    generate_global_lists(dictionary, global_lists, global_info)

new_dicts = []
# print (len(tables_list))
for i in range(0,len(tables_list)):
    dictionary = tables_list[i]
    preprocesStudio(dictionary, global_lists)
    try:
        dictionary = constriants(dictionary)
    except Exception as e:
        # print (dictionary["Tablename"].val)
        print("error in constraints: ",e)
        continue
    new_dicts.append(dictionary)
    num_of_fake_per_table = 5
    for index in range(0,num_of_fake_per_table):
        new_dict = table_generator(dictionary, global_lists, global_info, index)
        try:
            new_dict = constriants(new_dict)
        except Exception as e:
            # print (new_dict["Tablename"].val)
            print("error in constraints: ",e)
            continue
        new_dicts.append(new_dict)

for i in range(0,len(new_dicts)):
    dictionary = new_dicts[i]
    generate_fake_lists(dictionary,fake_adverse_list,global_info)

for i in range(0, len(new_dicts)):
#     print("------------------ " + str(i) + "---------------------")
    splitDateRange(new_dicts[i])
    preprocessDate(new_dicts[i])  
    
serial_ctr = 1
for i in range(0, len(new_dicts)):    
    dictionary = new_dicts[i]
    for key, value in dictionary.items():
        print( str(key) + ":" +  str(value.val) + str(value.tag) + "\n" )
    
    table_name = (dictionary["Tablename"].val).split("_")[-1]

    table = []    
    premises = []
    hypothesis = []
    label = []
    premises_used = []
    json_name = []
    #get json
    json_table = get_json(dictionary)
    json_sent = []
    for key in json_table:
        json_sent.append(json_table[key][0])
    json_sentences = ". ".join(json_sent)
    #length rules
#     print ("length_rules")
#     final_result = final_result + lengthRules(dictionary)
    #producer
    final_result = {}
    print ("Producer")
    final_result["Producer"] = []
    final_result["Producer"] = commonRules(dictionary, fake_adverse_list[table_name], 1)
#   label
    print ("Label")
    final_result["Label"] = []
    final_result["Label"] = commonRules(dictionary, fake_adverse_list[table_name], 2)
#   genre
    print ("Genre")
    final_result["Genre"] = []
    final_result["Genre"] = commonRules(dictionary, fake_adverse_list[table_name], 3)
#   studio
    print ("Studio")
    final_result["Studio"] = []
    final_result["Studio"] = commonRules(dictionary, fake_adverse_list[table_name], 4)
    
#     if final_result:
    for key in final_result:
        for statement in final_result[key]:
            truth_val = statement.split()[-1].strip()
            length = len(statement.split())
            statement = " ".join((statement.split()[:length -1])).strip()
            if truth_val == "True":
                truth_val = 1
            else:
                truth_val = 0
            table_name_entry = dictionary["Tablename"].val
            table_names_list = table_name_entry.split("_")
            table.append(table_names_list[0])
            json_name.append(table_names_list[1])
            premises.append(json_sentences)
            hypothesis.append(statement)
            premises_used.append(json_table[key])
            label.append(str(truth_val))
            
#             outputFile.write(str(serial_ctr) + "\t" + table_names_list[0] + "\t" + table_names_list[1] + "\t" + json_table + "\t" + statement + "\t" + str(truth_val) + "\n")
#             serial_ctr  += 1
            
    #date rules
#     print ("dates_rules")
#     final_result = []
#     final_result = final_result + dateRules(dictionary)
#     if final_result:
#         for statement in final_result:
#             truth_val = statement.split()[-1].strip()
#             length = len(statement.split())
#             statement = " ".join((statement.split()[:length -1])).strip()
#             if truth_val == "True":
#                 truth_val = 1
#             else:
#                 truth_val = 0
#             table_name_entry = dictionary["Tablename"].val
#             table_names_list = table_name_entry.split("_")
#             outputFile.write(str(serial_ctr) + "\t" + table_names_list[0] + "\t" + table_names_list[1] + "\t" + json_table + "\t" + statement + "\t" + str(truth_val) + "\n")
#             serial_ctr  += 1
    
df = pd.DataFrame({"table":table,"premises":premises,"hypothesis":hypothesis,"label":label, "key & premises_used":premises_used,"json_name":json_name})
df.to_csv("album_"+".csv",sep="\t")      

Title:In RainbowsNone

Released:10 Oct 2007DATE

Recorded:['5 February 2005', '8 June 2007']DATE

Genre:['Alternative rock', 'art rock', 'experimental rock', 'art pop']None

Length:42:39CARDINAL

Label:['Self-released', 'Xurbia Xendless', 'XL', 'TBD']None

Producer:['Nigel Godrich']PERSON

Tablename:T0_T0ID

In Rainbows
Producer
Label
Genre
Studio


TypeError: 'list' object is not callable